#### Parte 1. Extracción de Productos del PDF


In [443]:
import pandas as pd

In [444]:
from pypdf import PdfReader
reader = PdfReader("3. FAC001.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text() # Extrae el texto del pdf 

In [445]:
#---- Para el segundo pdf -------
reader2 = PdfReader("4. FAC002.pdf")
number_of_pages2 = len(reader2.pages)
page2 = reader2.pages[0]
text2 = page2.extract_text()

In [446]:
# Dividir la cadena en columna y valor
texto_dividido = text.split('\n')
for i in range(0,len(texto_dividido)):
    texto_dividido[i] = texto_dividido[i].split(" ")
texto_dividido[3][1] = texto_dividido[3][1] + " " + texto_dividido[3][2]

# Dividir la cadena en columna y valor
texto_dividido2 = text2.split('\n')
for i in range(0,len(texto_dividido2)):
    texto_dividido2[i] = texto_dividido2[i].split(" ")
    
texto_dividido2[4][1] = texto_dividido2[4][1] + " " + texto_dividido2[4][2]

In [447]:
datos_limpios = [[elemento for elemento in lista if elemento != 'Industrial'] for lista in texto_dividido]
datos_limpios2 = [[elemento for elemento in lista if elemento != 'solar'] for lista in texto_dividido2]

In [448]:
datos_limpios2 

[['numero_factura', 'FAC002'],
 ['fecha_compra', '15/01/2025'],
 ['codigo_producto', 'nombre_producto', 'cantidad'],
 ['PB03', 'Desodorante', '15'],
 ['PB04', 'Protector solar', '7']]

In [449]:
df = pd.DataFrame(datos_limpios[3:], columns=datos_limpios[2])
df_segundopdf = pd.DataFrame(datos_limpios2[3:], columns=datos_limpios2[2])

In [450]:
df_segundopdf

,codigo_producto,nombre_producto,cantidad
0,PB03,Desodorante,15
1,PB04,Protector solar,7


In [451]:
df

,codigo_producto,nombre_producto,cantidad
0,PB01,Alcohol Industrial,30
1,PB02,Shampoo,10


In [452]:
columna = []
valor = []
for elemento in datos_limpios[:2]:
    columna.append(elemento[0])
    if elemento[1]:
        valor.append(elemento[1])
columna2 = []
valor2 = []
for elemento in datos_limpios2[:2]:
    columna2.append(elemento[0])
    if elemento[1]:
        valor2.append(elemento[1])


In [453]:
df_tabla_chiquis = pd.DataFrame([valor]*2, columns=columna)
df_tabla_chiquis2 = pd.DataFrame([valor2]*2, columns=columna2)

In [454]:
# Unir ambas tablas por índice
db_completo_FAC001 = pd.merge(df_tabla_chiquis,df,left_index=True, right_index=True)
db_completo_FAC002 = pd.merge(df_tabla_chiquis2,df_segundopdf ,left_index=True, right_index=True)

In [465]:
db_pdfs_unidos = pd.concat([db_completo_FAC001,db_completo_FAC002])
db_pdfs_unidos['fecha_compra'] = pd.to_datetime(db_pdfs_unidos['fecha_compra'], format='%d/%m/%Y')

In [504]:
db_pdfs_unidos.iloc[:,0:3]

,numero_factura,fecha_compra,codigo_producto
0,FAC001,2025-01-21,PB01
1,FAC001,2025-01-21,PB02
0,FAC002,2025-01-15,PB03
1,FAC002,2025-01-15,PB04


In [508]:
db_pdfs_unidos.reset_index(drop=True)

,numero_factura,fecha_compra,codigo_producto,nombre_producto,cantidad
0,FAC001,2025-01-21,PB01,Alcohol Industrial,30
1,FAC001,2025-01-21,PB02,Shampoo,10
2,FAC002,2025-01-15,PB03,Desodorante,15
3,FAC002,2025-01-15,PB04,Protector solar,7


In [538]:
# Exportación datos de excel
db_facturas = pd.read_excel('2. BASE_DATOS_FACTURAS.xlsx')

In [540]:
actualizacion = pd.merge(db_facturas.iloc[0:4, 0:3],db_pdfs_unidos.iloc[:,0:3], on = 'codigo_producto')
db_facturas = pd.concat([actualizacion, db_facturas.iloc[4:6]])

In [541]:
db_facturas

,codigo_producto,nombre_producto,cantidad,numero_factura,fecha_compra
0,PB01,Alcohol Industrial,30,FAC001,2025-01-21
1,PB02,Shampoo,10,FAC001,2025-01-21
2,PB03,Desodorante,15,FAC002,2025-01-15
3,PB04,Protector solar,7,FAC002,2025-01-15
4,PB05,Talco Blanco,40,FAC003,2024-07-12
5,PB06,Perfume,10,FAC003,2024-07-12


In [542]:
db_facturas =  db_facturas [["codigo_producto", "nombre_producto", "cantidad", "fecha_compra", "numero_factura"]]

In [543]:
db_facturas 

,codigo_producto,nombre_producto,cantidad,fecha_compra,numero_factura
0,PB01,Alcohol Industrial,30,2025-01-21,FAC001
1,PB02,Shampoo,10,2025-01-21,FAC001
2,PB03,Desodorante,15,2025-01-15,FAC002
3,PB04,Protector solar,7,2025-01-15,FAC002
4,PB05,Talco Blanco,40,2024-07-12,FAC003
5,PB06,Perfume,10,2024-07-12,FAC003


In [544]:
type(db_facturas['numero_factura'][0])

str

In [545]:
db_facturas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   codigo_producto  6 non-null      object        
 1   nombre_producto  6 non-null      object        
 2   cantidad         6 non-null      int64         
 3   fecha_compra     6 non-null      datetime64[ns]
 4   numero_factura   6 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 372.0+ bytes
